In [ ]:
# 1 - slow

In [44]:
class LRUCacheEntry():
    def __init__(self, key, value, time):
        self.key = key
        self.value = value
        self.time = time

    def __gt__(e1, e2):
        return e1.time > e2.time

    def __ge__(e1, e2):
        return e1.time >= e2.time

    def __eq__(e1, e2):
        return e1.time == e2.time

    def __repr__(self):
        return '(key=%d; value=%d; time=%d)' % (self.key, self.value,
                                                self.time)


class LRUCache(object):
    def __init__(self, capacity):
        """
        :type capacity: int
        """

        self._time = 0
        self._capacity = capacity
        self._items = {}  # [{key,value,time}]

    def get(self, key):
        """
        :type key: int
        :rtype: int
        """

        if key in self._items:
            self._time += 1
            self._items[key].time = self._time
            return self._items[key].value
        else:
            return -1

    def put(self, key, value):
        """
        :type key: int
        :type value: int
        :rtype: void
        """

        self._time += 1

        if not key in self._items:

            self._items[key] = LRUCacheEntry(key, value, self._time)

        else:

            self._items[key].time = self._time
            self._items[key].value = value

        # remove least accesses item
        if len(self._items) > self._capacity:
            mintime = self._time
            key_to_remove = key

            for k in self._items:
                v = self._items[k]
                if v.time < mintime:
                    mintime = v.time
                    key_to_remove = k

            del self._items[key_to_remove]

In [ ]:
# 2 - with collections.OrderedDict

In [88]:
from collections import OrderedDict


class LRUCache(object):
    def __init__(self, capacity):
        """
        :type capacity: int
        """
        self._items = OrderedDict()
        self._capacity = capacity

    def get(self, key):
        """
        :type key: int
        :rtype: int
        """

        # reinsert at the end with same value
        if key in self._items:
            r = self._items.pop(key)
            self._items[key] = r
            return r

        return -1

    def put(self, key, value):
        """
        :type key: int
        :type value: int
        :rtype: void
        """

        # (re)insert at the end with new value

        if key in self._items:
            self._items.pop(key)

        self._items[key] = value

        # remove first (oldest) item when capacity exceeded
        if len(self._items) > self._capacity:
            self._items.popitem(last=False)

In [ ]:
# 3: Using doubly linked list

In [51]:
class DoublyLinkedNode():
    def __init__(self, value=None, key=None, prev=None, next=None):
        self.value = value
        self.key = key
        self.prev = prev
        self.next = next

    def __repr__(self):
        return '(key=%s, value=%s)' % (self.key, self.value)


class LRUCache():
    def __init__(self, capacity):
        self._capacity = capacity
        self._head = DoublyLinkedNode(key='head')  # newest
        self._tail = DoublyLinkedNode(key='tail')  # oldest
        self._head.next = self._tail
        self._head.prev = self._tail
        self._tail.prev = self._head
        self._tail.next = self._head

        self._nodes = {}

    def get(self, key):
        if key in self._nodes:
            self.put(key, self._nodes[key].value)
            return self._nodes[key].value

        return -1

    def put(self, key, value):

        if key not in self._nodes:

            # create new node
            node = DoublyLinkedNode(value=value, key=key)
            self._nodes[key] = node

        else:

            # update node val
            node = self._nodes[key]
            node.value = value

            # remove node
            node.prev.next = node.next
            node.next.prev = node.prev


        # insert before head
        node.next = self._head
        node.prev = self._head.prev
        self._head.prev.next = node
        self._head.prev = node
        

        # remove element right after tail if cache capacity exceeded
        if len(self._nodes) > self._capacity:
            e = self._tail.next
            e.prev.next = e.next
            e.next.prev = e.prev
            del self._nodes[e.key]

    def __repr__(self):

        r = ''
        n = self._tail

        while True:
            r += '(key=%s, value=%s)' % (n.key, n.value)

            if n is self._head:
                r += ' <-> [' + str(n.next.key) + ']'
                break

            r += ' <-> '
            n = n.next

        return r

In [57]:
c = LRUCache(2)

In [58]:
c.put(11, 111)
c.put(22, 222)

In [59]:
c

(key=tail, value=None) <-> (key=11, value=111) <-> (key=22, value=222) <-> (key=head, value=None) <-> [tail]

In [60]:
c.put(11, 1)

In [61]:
c

(key=tail, value=None) <-> (key=22, value=222) <-> (key=11, value=1) <-> (key=head, value=None) <-> [tail]

In [ ]:
c.put(22, 22222)

In [63]:
c

(key=tail, value=None) <-> (key=11, value=1) <-> (key=22, value=22222) <-> (key=head, value=None) <-> [tail]

In [64]:
c.put(33, 333)

In [65]:
c

(key=tail, value=None) <-> (key=22, value=22222) <-> (key=33, value=333) <-> (key=head, value=None) <-> [tail]

In [47]:
cache = LRUCache(2)

cache.put(1, 1)

cache.put(2, 2)
# print(cache._items, cache._indexes)

print(cache.get(1))
# returns 1

for i in cache._items:
    print(cache._items[i])

# print(cache._items, cache._indexes)

cache.put(3, 3)
# evicts key 2
# print(cache._items, cache._indexes)

print(cache.get(2))
# returns -1 (not found)

print(cache._items, cache._indexes, cache._deleted)

cache.put(4, 4);    # evicts key 1
# print(cache._items, cache._indexes)

print(cache.get(1));       # returns -1 (not found)
# print(cache._items, cache._indexes)

print(cache.get(3));       # returns 3

print(cache.get(4));       # returns 4
# # print(cache._items)